Fetch information about a specific person identified by his/her ORCID ID, and handle API errors.

In [223]:
import sys
sys.path.append('../')

# !{sys.executable} -m pip install matplotlib

import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import ast
import altair as alt
import networkx as nx
import nx_altair as nxa
import pandas as pd
from datetime import datetime, date
import requests
import json

import os
from os.path import join, dirname
from dotenv import load_dotenv
load_dotenv();

### ORCID ID not found

In [224]:
# ORCID ID not found
API_KEY = os.environ.get("API_KEY")
ORCID = "0000-0003-XXXX-XXXX"

url = f'https://f130.azure-api.net/v1/orcid/{ORCID}?subscription-key={API_KEY}'
r = requests.get(url)

# print a short confirmation on completion
print('Augment API query complete ', r.status_code)

if r.status_code == 400:
    print(r.json()["message"])

Augment API query complete  400
FAILED: Invalid ORCID 0000-0003-XXXX-XXXX


### Missing API_KEY

In [225]:
# Missing API_KEY
API_KEY = ''
ORCID = "0000-0002-4259-9774"

url = f'https://f130.azure-api.net/v1/orcid/{ORCID}?subscription-key={API_KEY}'
r = requests.get(url)

# print a short confirmation on completion
print('Augment API query complete ', r.status_code)

if r.status_code == 401:
    print(f'Authentication error.')

Augment API query complete  401
Authentication error.


### ORCID ID does exist

In [226]:
# ORCID ID does exist
API_KEY = os.environ.get("API_KEY")
ORCID = "0000-0002-4259-9774"

url = f'https://f130.azure-api.net/v1/orcid/{ORCID}?subscription-key={API_KEY}'
r = requests.get(url)

# print a short confirmation on completion
print('Augment API query complete ', r.status_code)

if r.status_code == 200 and r.json()[0]["nodes"]["researchers"]:    
    researchers = r.json()[0]["nodes"]["researchers"]
    
    researcher = None
    for i in range(len(researchers)):
        if researchers[i]["orcid"] == ORCID:
            researcher = researchers[i]

print()
print(f'ORCID: {researcher["orcid"]}')
print(f'First name: {researcher["first_name"]}')
print(f'Last name: {researcher["last_name"]}')
print()
print(f'The researcher {researcher["full_name"]} is connected to {r.json()[0]["stats"]}.')

Augment API query complete  200

ORCID: 0000-0002-4259-9774
First name: Amir
Last name: Aryani

The researcher Amir Aryani is connected to {'datasets': 0, 'grants': 3, 'organisations': 102, 'publications': 133, 'researchers': 56}.


### List of publications
Only includes publications with DOIs, and duplicate versions removed, sorted by publication year in descending order

In [227]:
# include all publications
pd.set_option("display.max_rows", None)

# Convert publication_year to int, use 9999 (an int) as NaN
def force_int(n):
    if n.get('publication_year'):
        n['publication_year'] = int(float(n['publication_year']))
        return n
    else:
        n['publication_year'] = 9999
        return n
    
json = map(force_int, r.json()[0]["nodes"]["publications"])
df = pd.DataFrame(json, columns=['doi', 'publication_year', 'title'])
df = df.dropna()
df = df.drop_duplicates(subset=['publication_year', 'title'])
df = df.sort_values(by=['publication_year','doi'], ascending=False)
dfStyler = df.style.set_properties(**{'text-align': 'left'})
dfStyler.set_table_styles([dict(selector='th', props=[('text-align', 'left')])])

,doi,publication_year,title
25,10.5281/zenodo.4953496,2021,Commuting-distance-by-income
24,10.5281/zenodo.4953493,2021,Commuting-distance-by-industry
26,10.5281/zenodo.4953492,2021,Commuting-distance-by-age
23,10.5281/zenodo.4953489,2021,Commuting-distance-by-occupation
22,10.5281/zenodo.4953486,2021,ILO-Youth-Unemployment
27,10.5281/zenodo.4939953,2021,People-with-disability-participated-in-labour-force-2018
75,10.5281/zenodo.4939952,2021,Population-of-people-having-a-disability
21,10.5281/zenodo.4939951,2021,12months-mental-disorders-by-age
28,10.5281/zenodo.4939949,2021,Mental-health-care-service-contact-rate
76,10.5281/zenodo.4939948,2021,Population-of-Australians-completed-year-12-in-2016


In [254]:
yf = df['publication_year'].groupby(df['publication_year']).agg('count').reset_index(name ='count')
print(yf)
alt.Chart(yf).mark_bar(color='green').properties(width=500).encode(
    alt.X("publication_year:O", bin="binned", axis=alt.Axis(title='Publication Year')),
    alt.Y("count", axis=alt.Axis(title='Number of Publications'))
).configure_axis(
    grid=False
).configure_view(
    strokeWidth=0
)

    publication_year  count
0               2009      1
1               2010      1
2               2011      1
3               2012      1
4               2013     10
5               2014      6
6               2015      5
7               2016      2
8               2017      7
9               2018      4
10              2019      5
11              2020      1
12              2021     21


alt.Chart(...)